In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkContext

SpSession = SparkSession \
    .builder \
    .appName("Pyspark Canary Turists") \
    .getOrCreate()

SpSession.sparkContext.setLogLevel("ERROR")

In [3]:
import pandas as pd
df_pd_known = pd.read_csv("https://datos.canarias.es/catalogos/estadisticas/dataset/f6c8c3ad-0426-4c7e-8fb5-93405e5ee981/resource/de9efe88-6f18-4e87-bebb-1621e654604a/download/c00028a_dsc_0113.csv")
df_sp_known = SpSession.createDataFrame(df_pd_known)
df_pd_money = pd.read_csv("https://datos.canarias.es/catalogos/estadisticas/dataset/f3271a16-acd7-44d4-840a-80179443f414/resource/199d47e1-72b6-423a-8b7a-9a9f3c4e1596/download/c00028a_dsc_0145.csv")
df_sp_money = SpSession.createDataFrame(df_pd_money)
df_pd_stay = pd.read_csv("https://datos.canarias.es/catalogos/estadisticas/dataset/04f2a069-17f0-44ea-8b61-3d998d7c305d/resource/82622785-6a8f-4db9-a520-794526ce957e/download/c00028a_dsc_0138.csv")
df_sp_stay = SpSession.createDataFrame(df_pd_stay)
df_pd_canal = pd.read_csv("https://datos.canarias.es/catalogos/estadisticas/dataset/eac981c4-ba6e-429f-a94d-2e3b6237b032/resource/4590bec9-9f7a-4344-b92d-da3c988cebde/download/c00028a_dsc_0119.csv")
df_sp_canal = SpSession.createDataFrame(df_pd_canal)


In [4]:
df_sp_known.show(5)
df_sp_money.show(5)

+-----------------------+--------------------+------------------------+--------+------------+-----------+--------------------+--------+-----------------+
|formas_conocer_Canarias|paises_residencia_id|paises_residencia_nombre|islas_id|islas_nombre|periodos_id|     periodos_nombre|turistas|notas_observacion|
+-----------------------+--------------------+------------------------+--------+------------+-----------+--------------------+--------+-----------------+
|   Conocía Canarias ...|                   T|            Total países|    ES70|    Canarias|     2006Q1|2006 Primer trime...| 1866124|              NaN|
|   Conocía Canarias ...|                   T|            Total países|    ES70|    Canarias|     2006Q2|2006 Segundo trim...| 1414361|              NaN|
|   Conocía Canarias ...|                   T|            Total países|    ES70|    Canarias|     2006Q3|2006 Tercer trime...| 1431374|              NaN|
|   Conocía Canarias ...|                   T|            Total países|    E

In [5]:
df_sp_stay.show(5)
df_sp_canal.show(5)

+--------------------+------------------------+------------------+--------+------------+-----------+--------------------+--------------+-----------------+
|paises_residencia_id|paises_residencia_nombre| tipos_alojamiento|islas_id|islas_nombre|periodos_id|     periodos_nombre|estancia_media|notas_observacion|
+--------------------+------------------------+------------------+--------+------------+-----------+--------------------+--------------+-----------------+
|                   T|            Total países|Total alojamientos|    ES70|    Canarias|     2006Q1|2006 Primer trime...|         10.28|              NaN|
|                   T|            Total países|Total alojamientos|    ES70|    Canarias|     2006Q2|2006 Segundo trim...|         10.08|              NaN|
|                   T|            Total países|Total alojamientos|    ES70|    Canarias|     2006Q3|2006 Tercer trime...|         10.47|              NaN|
|                   T|            Total países|Total alojamientos|    

In [6]:
df_sp_known.describe().show()

+-------+-----------------------+--------------------+------------------------+--------+------------+------------------+--------------------+------------------+--------------------+
|summary|formas_conocer_Canarias|paises_residencia_id|paises_residencia_nombre|islas_id|islas_nombre|       periodos_id|     periodos_nombre|          turistas|   notas_observacion|
+-------+-----------------------+--------------------+------------------------+--------+------------+------------------+--------------------+------------------+--------------------+
|  count|                  14700|               14700|                   14700|   14700|       14700|             14700|               14700|             14700|               14700|
|   mean|                   NULL|                NULL|                    NULL|    NULL|        NULL|            2010.5|              2008.5|115202.76655733128|                 NaN|
| stddev|                   NULL|                NULL|                    NULL|    NULL|  

In [7]:
# df_spark.select("formas_conocer_Canarias").distinct().show(truncate=False)
df_sp_known.groupBy("formas_conocer_Canarias").count().show(truncate=False)

+--------------------------------------------+-----+
|formas_conocer_Canarias                     |count|
+--------------------------------------------+-----+
|Conocía Canarias por visitas anteriores     |2100 |
|Recomendación de amigos o familiares        |2100 |
|Folleto o catálogo del tour operador        |2100 |
|Recomendación de agencia de viajes          |2100 |
|La información obtenida a través de Internet|2100 |
|Otros                                       |2100 |
|No contesta                                 |2100 |
+--------------------------------------------+-----+



#### De donde vienen los turistas que gastan mas dinero en las islas y a que islas van?

In [11]:
from pyspark.sql.functions import *
from pyspark.sql.types import *


In [17]:
df_sp_money.columns

['indicadores_gasto_id',
 'indicadores_gasto_nombre',
 'paises_residencia_id',
 'paises_residencia_nombre',
 'islas_id',
 'islas_nombre',
 'indicadores',
 'periodos_id',
 'periodos_nombre',
 'macromagnitudes',
 'notas_observacion']

In [32]:
df_money = df_sp_money.withColumn(
    "macromagnitudes_num",
    when(col("macromagnitudes").rlike(r"^-?\d+(\.\d+)?$"), col("macromagnitudes").cast("double"))
    .otherwise(None)
)

In [35]:
df_money.dtypes

[('indicadores_gasto_id', 'string'),
 ('indicadores_gasto_nombre', 'string'),
 ('paises_residencia_id', 'string'),
 ('paises_residencia_nombre', 'string'),
 ('islas_id', 'string'),
 ('islas_nombre', 'string'),
 ('indicadores', 'string'),
 ('periodos_id', 'string'),
 ('periodos_nombre', 'string'),
 ('macromagnitudes', 'string'),
 ('notas_observacion', 'string'),
 ('macromagnitudes_num', 'double')]

In [36]:
df_money.select("macromagnitudes_num").distinct().show()

+-------------------+
|macromagnitudes_num|
+-------------------+
|     2.0721110797E9|
|     3.7214021988E9|
|               2.86|
|     5.4217550881E8|
|              17.95|
|              26.72|
|       9.06790442E8|
|        1.6106505E8|
|             -25.86|
|     5.8728029399E8|
|     2.2557389016E8|
|     2.0515863006E8|
|               14.9|
|              82.51|
|               9.13|
|              23.04|
|          992157.85|
|     3.3387216152E8|
|     8.3211742905E8|
|              19.27|
+-------------------+
only showing top 20 rows


In [40]:
df_money_l = df_money.filter(col("paises_residencia_nombre") != "Total países")


In [41]:
df_money_l.select("paises_residencia_nombre").distinct().show()

+------------------------+
|paises_residencia_nombre|
+------------------------+
|                  España|
|         Países Nórdicos|
|                Alemania|
|                 Holanda|
|            Otros países|
|             Reino Unido|
+------------------------+



In [42]:
gasto_por_pais = df_money_l.groupBy("paises_residencia_nombre") \
    .agg(
        avg("macromagnitudes_num").alias("gasto_promedio"),
        sum("macromagnitudes_num").alias("gasto_total"),
        count("*").alias("num_registros")
    ) \
    .orderBy(desc("gasto_promedio"))

print("Top 10 países por gasto promedio:")
gasto_por_pais.show(10)

Top 10 países por gasto promedio:
+------------------------+--------------------+--------------------+-------------+
|paises_residencia_nombre|      gasto_promedio|         gasto_total|num_registros|
+------------------------+--------------------+--------------------+-------------+
|             Reino Unido|1.1019812127841346E8|2.611695474298399E11|         2700|
|                Alemania| 9.483528379998337E7|2.287427045255598...|         2700|
|            Otros países| 7.854427843284823E7|1.894487995800299...|         2700|
|         Países Nórdicos| 5.462620341312772E7|1.257495202570200...|         2700|
|                  España| 4.502005406478439E7|1.085883704042599...|         2700|
|                 Holanda|1.5686055913123671E7|3.741124335279996E10|         2700|
+------------------------+--------------------+--------------------+-------------+



In [ ]:
### Que quiero responder?

#### Hay relacion entre el gasto y el tiempo de estadia? es decir, si se quedan mas tiempo gastan mas?
#### 